<a href="https://colab.research.google.com/github/DA24B020/Project-SMAC/blob/main/Ayush/Finance/stochastic_volatility_without_leverage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving FTSE100_data.csv to FTSE100_data.csv


In [ ]:
import pandas as pd

# Reading the file
df = pd.read_csv("FTSE100_data.csv")
print(df.head())
print(df.columns)

         Date   Price    Open    High     Low  Vol. Change %
0  06-05-2008  6215.2  6215.5  6233.7  6155.9   NaN    0.00%
1  02-05-2008  6215.5  6087.3  6223.9  6087.3   NaN    2.11%
2  01-05-2008  6087.3  6087.3  6118.2  6066.0   NaN    0.00%
3  30-04-2008  6087.3  6089.4  6120.3  6035.8   NaN   -0.03%
4  29-04-2008  6089.4  6090.4  6133.5  6051.6   NaN   -0.02%
Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %'], dtype='object')


In [ ]:
# Parse with day first
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

# Sort so oldest date is first
df = df.sort_values('Date').reset_index(drop=True)


In [ ]:
# Parse with day first
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

# Sort so oldest date is first
df = df.sort_values('Date').reset_index(drop=True)


In [ ]:
print(df.head())
print(df.columns)

        Date   Price    Open    High     Low  Vol. Change %
0 2001-01-03  6039.9  6174.7  6174.7  6029.3   NaN   -2.18%
1 2001-01-04  6185.6  6039.9  6195.3  6039.9   NaN    2.41%
2 2001-01-05  6198.1  6185.6  6239.6  6155.0   NaN    0.20%
3 2001-01-08  6149.6  6198.1  6212.4  6137.7   NaN   -0.78%
4 2001-01-09  6088.1  6149.6  6195.9  6066.4   NaN   -1.00%
Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %'], dtype='object')


In [ ]:
# Convert the 'Price' column to numeric values.
# Some CSV files (e.g., from Investing.com) store numbers as text with commas for thousands, e.g., "6,543.21".
# .replace(',', '', regex=True) removes commas,
# pd.to_numeric converts strings to floats (numeric type).
df['Price'] = pd.to_numeric(df['Price'].replace(',', '', regex=True))

# Removing missing values
df = df.dropna(subset=['Price', 'Date'])

import numpy as np

# Extract the 'Price' column values as a NumPy array for fast math operations
prices = df['Price'].values

# Compute daily log returns multiplied by 100 to express in percentage points.
# Formula: x_t = 100 * log(s_t / s_{t-1}), where s_t is today's price.
# np.diff(log(prices)) calculates log(s_t) - log(s_{t-1}) for each consecutive day.
returns = 100 * np.diff(np.log(prices))


# Print maximum daily return in the dataset (largest positive percentage move).
print("Max return:", returns.max())

# Print minimum daily return in the dataset (largest negative percentage move).
print("Min return:", returns.min())

Max return: 5.903779465355363
Min return: -5.6374301294075835


In [ ]:
# Check if there are any NaN (Not-a-Number) values in the returns array.
# np.isnan(returns) creates a boolean array marking True for each NaN entry.
# np.any(...) returns True if at least one element in that boolean array is True,
# meaning there is at least one NaN in returns; otherwise it returns False.
print(np.any(np.isnan(returns)))

False


In [ ]:
# --- Initial parameter guesses for the SV model ---
beta_init = 0.866    # Initial guess for β (scale of volatility) — controls overall volatility level
phi_init  = 0.963    # Initial guess for φ (persistence of log-volatility, AR(1) coefficient)
sigma_init = 0.16    # Initial guess for σ (volatility-of-volatility, standard deviation of g_t innovations)

# --- Discretization setup for latent log-volatility g_t ---
m_states = 50        # Number of discrete states used to approximate continuous g_t (HMM discretization)
g_min = -2.5         # Lower bound of g_t range for discretization
g_max =  2.5         # Upper bound of g_t range for discretization

In [ ]:
# Define the bin edges for discretizing g_t
# Splits the range [g_min, g_max] into m_states equal intervals: b0, b1, ..., bm
b_edges = np.linspace(g_min, g_max, m_states + 1)

# Compute the midpoint of each interval
# This midpoint g*_i will represent all continuous g_t values in that bin
g_mid = 0.5 * (b_edges[:-1] + b_edges[1:])

from scipy.stats import norm  # for Gaussian CDF

# Build the transition probability matrix Gamma
# Gamma[i, j] = P(next g_t in bin j | current g_t in bin i)
# Approximated by AR(1) Gaussian distribution of g_t+1 with mean = phi * g_mid[i], variance = sigma^2
def build_gamma(phi, sigma):
    # Initialize m_states x m_states matrix
    Gamma = np.zeros((m_states, m_states))

    for i in range(m_states):
        # Mean of g_{t+1} conditional on current state's midpoint
        mu_next = phi * g_mid[i]

        # Probability mass for g_{t+1} falling into each bin j
        # Difference of CDF values over bin j's edges
        Gamma[i, :] = norm.cdf((b_edges[1:]   - mu_next) / sigma) - \
                      norm.cdf((b_edges[:-1] - mu_next) / sigma)
    return Gamma

# Example usage — build the transition matrix for given phi and sigma
Gamma = build_gamma(phi_init, sigma_init)
# Check dimensions — should be (m_states x m_states)
print(Gamma.shape)  # For m_states = 50, prints (50, 50)


(50, 50)


In [ ]:
def stationary_dist(Gamma):
    """
    Compute stationary distribution δ from transition matrix Γ.
    δ satisfies δΓ = δ and sum(δ) = 1.

    Parameters
    ----------
    Gamma : ndarray (m_states x m_states)
        Transition probability matrix of the Markov chain.

    Returns
    -------
    delta : ndarray (m_states,)
        Stationary distribution vector.
    """
    m_states = Gamma.shape[0]

    # Build the system of equations:
    # (I - Γ^T) δ = 0  (stationary condition: δΓ = δ)
    # sum(δ) = 1
    A = np.vstack((np.eye(m_states) - Gamma.T, np.ones(m_states)))
    b = np.append(np.zeros(m_states), 1)

    # Solve in least-squares sense for δ
    delta = np.linalg.lstsq(A, b, rcond=None)[0]

    return delta

def emission_probs(obs, beta, g_mid):
    """
    Compute emission probabilities P(x_t | state i) for all states.

    Parameters
    ----------
    obs : float
        Observed return at time t.
    beta : float
        Scale parameter of SV model.
    g_mid : ndarray
        Midpoints (g*_i) of discretized log-volatility states.

    Returns
    -------
    probs : ndarray
        Probability density of `obs` for each state i.
    """
    # State-dependent variances: β² * exp(g*_i)
    state_variances = beta**2 * np.exp(g_mid)

    # PDF of Normal(0, variance) for each state
    return norm.pdf(obs, loc=0, scale=np.sqrt(state_variances))

def build_gamma(phi, sigma, b_edges, g_mid):
    """
    Build transition probability matrix Γ for discretized AR(1) log-volatility process.

    Parameters
    ----------
    phi : float
        AR(1) persistence parameter |phi| < 1.
    sigma : float
        Std dev of innovations to log-volatility (vol-of-vol).
    b_edges : ndarray
        Bin edges for discretization of g_t state space (length m_states+1).
    g_mid : ndarray
        Midpoints of each state interval (length m_states).

    Returns
    -------
    Gamma : ndarray (m_states x m_states)
        Transition probability matrix, where Gamma[i, j] is the
        probability of moving from state i to state j in one time step.
    """
    m_states = len(g_mid)
    Gamma = np.zeros((m_states, m_states))

    for i in range(m_states):
        # Mean of g_{t+1} given g_t is at midpoint of state i
        mu_next = phi * g_mid[i]

        # Probability g_{t+1} falls into each bin j given current state i
        Gamma[i, :] = norm.cdf((b_edges[1:]   - mu_next) / sigma) - \
                      norm.cdf((b_edges[:-1] - mu_next) / sigma)

    return Gamma

def sv_hmm_negloglik(params, x, b_edges, g_mid):
    beta, phi, sigma = params
    if beta <= 0 or abs(phi) >= 1 or sigma <= 0:
        return 1e10
    Gamma = build_gamma(phi, sigma, b_edges, g_mid)  # your existing function
    delta = stationary_dist(Gamma)
    # log of delta and emission for time 0
    log_alpha = np.log(delta + 1e-300) + np.log(emission_probs(x[0], beta, g_mid) + 1e-300)
    for t in range(1, len(x)):
        # Expand log_alpha from shape (m,) to (m,1) for broadcast
        # Add log transition probabilities: log(Gamma) shape (m, m)
        temp = log_alpha[:, None] + np.log(Gamma + 1e-300)

        # log-sum-exp across rows for each next state to find log_alpha for next step
        max_temp = np.max(temp, axis=0)
        log_alpha = max_temp + np.log(np.sum(np.exp(temp - max_temp), axis=0) + 1e-300)

        # Add log of emission probabilities for observation at time t
        log_alpha += np.log(emission_probs(x[t], beta, g_mid) + 1e-300)

    # Final log likelihood is log-sum-exp of log_alpha vector
    max_log_alpha = np.max(log_alpha)
    log_likelihood = max_log_alpha + np.log(np.sum(np.exp(log_alpha - max_log_alpha)))

    return -log_likelihood

# Initial parameter guess
init_params = [0.866, 0.963, 0.16]

from scipy.optimize import minimize
result = minimize(
    sv_hmm_negloglik, init_params,
    args=(returns, b_edges, g_mid),
    method='L-BFGS-B',
    bounds=[(1e-5, None), (-0.999, 0.999), (1e-5, None)],
    options={'disp': True}
)

print("Estimated parameters (beta, phi, sigma):", result.x)
print("Negative log-likelihood:", result.fun)

/tmp/ipython-input-12803671.py:117: DeprecationWarning: scipy.optimize: The `disp` and `iprint` options of the L-BFGS-B solver are deprecated and will be removed in SciPy 1.18.0.
  result = minimize(


Estimated parameters (beta, phi, sigma): [0.9938813  0.98842213 0.14318663]
Negative log-likelihood: 2500.9268687066797
